In [28]:
import numpy as np
import os
import torch

In [1]:
dir = "C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/1_1/states"

In [9]:
feature_indices = [(i, i + 1) for i in range(24) if (i % 3 == 0)]

In [3]:
array_list = []
for file in os.listdir(dir):
    f = os.path.join(dir, file)
    states = np.load(f, allow_pickle=True)
    array_list.append(states)

full = np.vstack(array_list)

In [4]:
matrix = full.copy()

In [7]:
def normalize(matrix):
    columns = matrix.shape[1]
    feature_indices = [(i, i + 1) for i in range(columns) if (i % 3 == 0)]
    min_max = []

    for i in feature_indices:
        min_max.append((np.min(matrix[:, i[0]]), np.max(matrix[:, i[0]])))
        min_max.append((np.min(matrix[:, i[1]]), np.max(matrix[:, i[1]])))

    for i in range(len(matrix)):
        index = 0
        for j in feature_indices:
            matrix[i, j[0]] = (
                    (matrix[i, j[0]] - min_max[index][0]) / (min_max[index][1] - min_max[index][0]))
            matrix[i, j[1]] = (
                    (matrix[i, j[1]] - min_max[index + 1][0]) / (min_max[index + 1][1] - min_max[index + 1][0]))
            index += 2
    return matrix

In [8]:
normalize(matrix)

array([[0.45454545, 0.11538462, 1.        , ..., 0.18181818, 0.        ,
        0.        ],
       [0.43181818, 0.07692308, 1.        , ..., 0.18181818, 0.        ,
        0.        ],
       [0.40909091, 0.03846154, 1.        , ..., 0.18181818, 0.        ,
        0.        ],
       ...,
       [0.43181818, 0.        , 0.        , ..., 0.09090909, 0.11764706,
        1.        ],
       [0.43181818, 0.        , 0.        , ..., 0.09090909, 0.05882353,
        1.        ],
       [0.43181818, 0.        , 0.        , ..., 0.09090909, 0.        ,
        1.        ]])

In [51]:
targets = "C:/Users/evani/OneDrive/AI leiden/Sanquin/RL_matching-main/NN training data/1_1/q_matrices"

In [73]:
array_list = []
for file in os.listdir(targets):
    f = os.path.join(targets, file)
    states = np.load(f, allow_pickle=True)
    array_list.append(states)

y = np.vstack(array_list)
y = torch.from_numpy(y).long()

In [91]:
def weighted_sampler(y):
    class_counts = torch.sum(y, dim=0)
    class_weights = 1. / class_counts
    class_weights_all = class_weights[y]

    # weighted_sampler = WeightedRandomSampler(
    #     weights=class_weights_all,
    #     num_samples=len(class_weights_all),
    #     replacement=True
    # )

    # return weighted_sampler

    return class_weights_all, class_weights

In [94]:
torch.sum(y, dim=0)

tensor([144240, 417620,  24599,  70498, 101236, 373874,   3944,   7213])

In [95]:
weights, counts = weighted_sampler(y)

tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [96]:
import torch.nn as nn

In [143]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [152]:
target = torch.empty(3, dtype=torch.long).random_(5)

In [153]:
target

tensor([0, 4, 4])

In [112]:
indices = (y == 1).nonzero()[:, 1]

In [187]:
from torch.utils.data import Dataset

In [209]:
class MyData(Dataset):
    def __init__(self, data_path=None, target_path=None):
        train_list = []
        for i in os.listdir(data_path):
            data = np.load(data_path + i, allow_pickle=True)
            train_list.append(data)

        test_list = []
        for i in os.listdir(target_path):
            data = np.load(target_path + i, allow_pickle=True)
            test_list.append(data)

        self.x = torch.from_numpy(np.vstack(train_list)).float()
        self.y = torch.from_numpy(np.vstack(test_list)).float()
        self.y = (self.y == 1).nonzero()[:, 1]

    def __getitem__(self, idx):
        sample = self.x[idx], self.y[idx]
        return sample

    def __len__(self):
        return len(self.x)

In [211]:
data_path = 'NN training data/1_1/states/'
target_path = 'NN training data/1_1/q_matrices/'

dataset = MyData(data_path, target_path)

In [222]:
targets = dataset.y

In [224]:
class_counts = torch.unique(targets, return_counts=True)[1]
class_weights = 1. / class_counts
class_weights_all = class_weights[targets]

In [226]:
len(class_weights_all)

1143224

In [227]:
class_weights

tensor([6.9329e-06, 2.3945e-06, 4.0652e-05, 1.4185e-05, 9.8779e-06, 2.6747e-06,
        2.5355e-04, 1.3864e-04])